# Spotify Listening History Analysis

![Spotify Image](spotify-logo.jpg)

## Author: PH Kaumbu
## Student Number: 220080995
## Source: Personal Data from Spotify
## Github Url: 

## Project Overview
This project aims to perform descriptive analysis on personal Spotify listening history. The objective is to uncover patterns, trends, and insights into my music listening habits over time. By analyzing my streaming history and library data provided by Spotify, I hope to gain a deeper understanding of my music preferences and how they have evolved.

## Data Description
The data comes in JSON format, divided into three main categories:
- **Streaming History:** Contains detailed records of songs listened to, including artist name, track name, end time, and listening duration. We have three files representing different time periods.
- **Library Information:** Provides information about the tracks saved in the Spotify library, including artist name, album name, and track details.

## Analysis Steps
1. **Data Loading:** Load the JSON files into pandas DataFrames for easy manipulation and analysis.
2. **Data Cleaning:** Inspect the data for inconsistencies, missing values, or anomalies that could skew the analysis. Standardize the data formats if necessary.
3. **Exploratory Data Analysis (EDA):**
   - Analyze listening trends over time.
   - Identify top artists, albums, and tracks.
   - Explore listening habits (e.g., most active listening times/days).
4. **Insights and Visualization:** Use graphs and charts to visualize findings from the EDA. Highlight any interesting patterns or insights about listening preferences and habits.
5. **Conclusions:** Summarize the key takeaways and any potential recommendations for future listening based on the analysis.

## Interesting Facts and Learnings
(As we progress through the analysis, this section will be populated with any interesting findings, patterns, or insights derived from the data.)


## 1. Data Loading
There is a requirements file in the same directory as this script
`pip install -r requirements.txt` to install the required packages


In [6]:
import pandas as pd
import os
import matplotlib.pyplot as plt

print(os.getcwd())  # To get a sense of where we are

# Load one of the streaming history files
streaming_history_path = "StreamingHistory_music_0.json"

# Load the data into a pandas dataframe
streaming_history_df = pd.read_json(streaming_history_path)

# Display the first few rows of the dataframe to understand its structure
streaming_history_df.head()

/Users/heritierkaumbu/Documents/Bcom Hons IS/Learning From Data/Spotify Account Data


,endTime,artistName,trackName,msPlayed
0,2023-03-10 22:58,Hot Chelle Rae,Don't Say Goodnight,119586
1,2023-03-11 05:20,Florocka,Twale,256998
2,2023-03-11 05:27,Banky W.,My Destiny,435472
3,2023-03-11 05:42,DOE,What I'm Waiting For,170589
4,2023-03-11 18:14,DJ Khaled,"All I Do Is Win (feat. T-Pain, Ludacris, Snoop...",232506


The first streaming history file has been successfully loaded, and here's a glimpse into its structure:

- **endTime:** The timestamp when the listening session ended.
- **artistName:** The name of the artist.
- **trackName:** The name of the track.
- **msPlayed:** The duration of the track played in milliseconds.

With this structure in mind, we can proceed to load all streaming history data and the library information to perform a comprehensive analysis. Let's move on to combining all streaming history data into a single DataFrame for a more extensive exploration.

In [7]:
# Paths to all streaming history files
streaming_files = [
    "StreamingHistory_music_0.json",
    "StreamingHistory_music_1.json",
    "StreamingHistory_music_2.json",
]

# Load and concatenate all streaming history data
all_streaming_df = pd.concat(
    [pd.read_json(file) for file in streaming_files], ignore_index=True
)

# Display basic information about the combined DataFrame
all_streaming_df.info()

# Preview the first few rows of the combined DataFrame
all_streaming_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28110 entries, 0 to 28109
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     28110 non-null  object
 1   artistName  28110 non-null  object
 2   trackName   28110 non-null  object
 3   msPlayed    28110 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 878.6+ KB


,endTime,artistName,trackName,msPlayed
0,2023-03-10 22:58,Hot Chelle Rae,Don't Say Goodnight,119586
1,2023-03-11 05:20,Florocka,Twale,256998
2,2023-03-11 05:27,Banky W.,My Destiny,435472
3,2023-03-11 05:42,DOE,What I'm Waiting For,170589
4,2023-03-11 18:14,DJ Khaled,"All I Do Is Win (feat. T-Pain, Ludacris, Snoop...",232506



Loaded the Spotify streaming history into a DataFrame, revealing 28,110 entries detailing my listening sessions. The data is clean, with each record showing when I finished listening to a track, who the artist was, the track name, and how long I listened.

### Initial Observations:

- All data is present and accounted for—no cleaning required.
- Memory usage is minimal, so we're good to handle the data in-memory.

### Next Steps:

The plan is to convert listening time to minutes, identify top artists and tracks, look for listening patterns over time, and present these insights visually.

Let's dive into the data and see what my music preferences reveal about me!



# 2.Exploratory Data Analysis (EDA)

In [8]:

# Convert 'msPlayed' to 'minutesPlayed' for better readability
all_streaming_df['minutesPlayed'] = all_streaming_df['msPlayed'] / 60000

# Calculate total listening time in minutes
total_listening_minutes = all_streaming_df['minutesPlayed'].sum()
print(f"Total Listening Time: {total_listening_minutes} minutes")

# Calculate the average listening time per track in minutes
average_track_length_minutes = all_streaming_df['minutesPlayed'].mean()
print(f"Average Track Length: {average_track_length_minutes:.2f} minutes")

# Find the most common artist - the mode for 'artistName'
most_common_artist = all_streaming_df['artistName'].mode()[0]
print(f"Most Common Artist: {most_common_artist}")

# Find the most common track - the mode for 'trackName'
most_common_track = all_streaming_df['trackName'].mode()[0]
print(f"Most Common Track: {most_common_track}")

# Standard Deviation for 'minutesPlayed' - measures the amount of variation or dispersion of listening times
std_dev_listening_times = all_streaming_df['minutesPlayed'].std()
print(f"Standard Deviation of Listening Times: {std_dev_listening_times:.2f} minutes")

# Median of 'minutesPlayed' - a better measure of central tendency when data is skewed
median_listening_time = all_streaming_df['minutesPlayed'].median()
print(f"Median Listening Time per Track: {median_listening_time:.2f} minutes")


Total Listening Time: 88115.27513333334 minutes
Average Track Length: 3.13 minutes
Most Common Artist: Hillsong Instrumentals
Most Common Track: My Blessings (Love Me)
Standard Deviation of Listening Times: 1.70 minutes
Median Listening Time per Track: 3.22 minutes


- **Total Listening Time:** 88,115.28 minutes spent on Spotify. That's about 61 days of music! It showcases the significant portion of time music occupies in my daily life.
  
- **Average Track Length:** At 3.13 minutes per track, it's clear that I favor songs of a typical length, suggesting a preference for standard, radio-friendly tracks.

- **Most Common Artist:** Hillsong Instrumentals topping the list indicates a strong inclination towards their music, that's because I'm a Chistian that code 😂. Helps with focus.

- **Most Common Track:** 'My Blessings (Love Me)' as the most frequently played track. I really loved that song. It resonates with me on a personal level, both for its melody or lyrics. I have been practicing gratefulness and appreciation for my life and the positive things going on in it.

- **Standard Deviation of Listening Times:** A standard deviation of 1.70 minutes points to a relatively consistent track length preference with some variety.

- **Median Listening Time per Track:** The median time of 3.22 minutes is close to the average, reinforcing that most tracks I listen to are around the standard song length.

Let’s break down these next steps:

1. **Temporal Analysis**: Identify any patterns based on the `endTime` column to see when I listen to music the most, possibly by extracting the hour of the day or the day of the week from the timestamps.
2. **Variability in Listening Sessions**: Use the standard deviation to explore how varied my listening sessions are. A higher standard deviation would indicate that the length of time I spend listening varies more significantly from session to session.
3. **Genre Exploration**: If available, categorize tracks by genre and analyze which genres I listen to most often, and see if there are shifts in genre preference over time.
4. **Engagement Over Time**: Calculate metrics like the total number of tracks listened per day or month to evaluate my engagement over the period represented by the data.


In [9]:


# Convert 'endTime' to datetime format and create new columns for day of the week and hour of the day
all_streaming_df['endTime'] = pd.to_datetime(all_streaming_df['endTime'])

all_streaming_df['dayOfWeek'] = all_streaming_df['endTime'].dt.day_name()
all_streaming_df['hourOfDay'] = all_streaming_df['endTime'].dt.hour

# Calculate the total listening time for each day of the week
total_listening_time_by_day = all_streaming_df.groupby('dayOfWeek')['minutesPlayed'].sum().sort_values()

# Calculate the mean listening time for each hour of the day
average_listening_time_by_hour = all_streaming_df.groupby('hourOfDay')['minutesPlayed'].mean().sort_values()

# Determine the most active listening day of the week
most_active_listening_day = total_listening_time_by_day.idxmax()

# Determine the most active listening hour of the day
most_active_listening_hour = average_listening_time_by_hour.idxmax()

# Calculate the variability in listening sessions across different hours of the day
variability_by_hour = all_streaming_df.groupby('hourOfDay')['minutesPlayed'].std()

# Find the median listening time by day of the week
median_listening_time_by_day = all_streaming_df.groupby('dayOfWeek')['minutesPlayed'].median().sort_values()

# Print out the calculated metrics
print(f"Total Listening Time by Day:\n{total_listening_time_by_day}\n")
print(f"Average Listening Time by Hour:\n{average_listening_time_by_hour}\n")
print(f"Most Active Listening Day: {most_active_listening_day}")
print(f"Most Active Listening Hour: {most_active_listening_hour} o'clock")
print(f"Variability in Listening Sessions by Hour:\n{variability_by_hour}\n")
print(f"Median Listening Time by Day:\n{median_listening_time_by_day}\n")


Total Listening Time by Day:
dayOfWeek
Sunday        9862.170800
Wednesday    10460.158783
Monday       12550.803483
Friday       12848.220717
Tuesday      13501.420617
Thursday     13524.394833
Saturday     15368.105900
Name: minutesPlayed, dtype: float64

Average Listening Time by Hour:
hourOfDay
17    2.735068
14    2.819591
19    2.858447
18    2.903232
16    2.910174
12    2.922905
15    2.945923
21    2.960521
10    2.992653
20    3.011407
13    3.013192
11    3.026278
7     3.117266
9     3.123484
6     3.160322
8     3.233287
22    3.349536
5     3.408720
23    3.468259
0     3.603213
4     3.623562
2     3.780599
1     3.791506
3     3.868385
Name: minutesPlayed, dtype: float64

Most Active Listening Day: Saturday
Most Active Listening Hour: 3 o'clock
Variability in Listening Sessions by Hour:
hourOfDay
0     1.452867
1     1.549547
2     1.497633
3     1.660310
4     1.829009
5     1.828476
6     1.903588
7     1.856959
8     1.899253
9     1.697958
10    1.599455
11    1.600


- **Total Listening Time by Day:** Saturdays, I listen to music the most. It fits with coding, sports, and chilling out.
  
- **Average Listening Time by Hour:** At 3 AM, it looks like I listen to a lot of music. This is when I often code at night on weekends.

- **Most Active Listening Day:** Saturday is the day I play the most music, which makes sense—it’s a busy day with lots of different activities.

- **Most Active Listening Hour:** The data shows I listen to music the most at 3 AM, which lines up with when I’m usually up late working on code.

- **Variability in Listening Sessions by Hour:** There's a big mix in how long I listen to music early in the morning. Some days it’s just for a quick workout or during a shower, and other times it’s for longer while coding.

- **Median Listening Time by Day:** On Fridays, I tend to have music on for longer times, which might be when I start my weekend coding and have a good workout with music.

So, music is a big part of my day, every day. It keeps me going whether I’m working out, coding, or just getting ready for the day. It’s cool to see how it fits into my daily life and studies.

### Top 10 listened to artists and tracks


In [10]:
# Calculate the top 10 most listened to artists and tracks
top_artists = all_streaming_df['artistName'].value_counts().head(10)
top_tracks = all_streaming_df['trackName'].value_counts().head(10)

# Group by hour and track name to identify preferences for specific activities
patterns_by_hour_track = all_streaming_df.groupby(['hourOfDay', 'trackName']).size().reset_index(name='count')
top_track_each_hour = patterns_by_hour_track.loc[patterns_by_hour_track.groupby('hourOfDay')['count'].idxmax()]

# Display the top artists and tracks
print("Top 10 Artists:\n", top_artists)
print("\nTop 10 Tracks:\n", top_tracks)
print("\nTop Track Each Hour:\n", top_track_each_hour[['hourOfDay', 'trackName', 'count']])


Top 10 Artists:
 artistName
Hillsong Instrumentals    1781
Mali Music                 637
Lecrae                     597
Hillsong Worship           569
Kanye West                 561
Michael W. Smith           502
Planetshakers              476
Matt Maher                 353
Don Moen                   277
Paul Wilbur                227
Name: count, dtype: int64

Top 10 Tracks:
 trackName
My Blessings (Love Me)                               305
Lord, I Need You                                     158
For Your Name Is Holy                                100
Because He Lives                                      74
Nothing Is Impossible (Featuring Israel Houghton)     69
Overtaken (From"Onepiece")                            63
Mighty To Save                                        61
Turn It Up - Live                                     58
Run                                                   56
Endless Praise - Live                                 55
Name: count, dtype: int64

Top Track Ea

- Favorite artists and tracks reflect my Christian faith, with Hillsong Instrumentals as my top artist.
- "My Blessings (Love Me)" by Mali Music is my most played track, significant for its motivational value.
- "Lord, I Need You" is a recurring early-hour favorite, likely part of my morning routine for focus or inspiration.
- Through the morning to early afternoon, "My Blessings (Love Me)" dominates, indicating its role in positive mornings.
- Evening variety with tracks like "Bound 2" by Kanye West shows a mix for winding down or energizing my nights.


### Skewness of Listening Times
Diving into the skewness to see how my music sessions tip the scale—do I favor shorter jams or the occasional long play?



In [11]:
# Manual skewness calculation
n = len(all_streaming_df['minutesPlayed'])
mean_listening_time = all_streaming_df['minutesPlayed'].mean()
sigma = all_streaming_df['minutesPlayed'].std()

# Calculate the sum of cubed deviations from the mean
sum_cubed_deviations = ((all_streaming_df['minutesPlayed'] - mean_listening_time) ** 3).sum()

# Skewness formula
skewness_manual = (n / ((n - 1) * (n - 2))) * (sum_cubed_deviations / sigma ** 3)

# Using Pandas to calculate skewness for comparison
skewness_ = all_streaming_df['minutesPlayed'].skew()

skewness_manual, skewness_


(0.4915925574038812, 0.49159255740388114)


The skewness leaning a bit to the right tells me my usual picks are on the quicker side, perfect for keeping the vibe going without dragging out too long. But hey, every now and then, a longer track finds its way into my heart (and my playlist)

### Median


In [13]:

# Manual Calculation
sorted_times = all_streaming_df['minutesPlayed'].sort_values().reset_index(drop=True)
num_observations = len(sorted_times)
is_even = num_observations % 2 == 0

if is_even:
    middle_index1 = num_observations // 2 - 1
    middle_index2 = num_observations // 2
    median_manual = (sorted_times[middle_index1] + sorted_times[middle_index2]) / 2.0
else:
    middle_index = num_observations // 2
    median_manual = sorted_times[middle_index]

print(f"Manually Calculated Median Listening Time: {median_manual:.2f} minutes")

# Using Pandas
median_pandas = all_streaming_df['minutesPlayed'].median()
print(f"Pandas Calculated Median Listening Time: {median_pandas:.2f} minutes")


Manually Calculated Median Listening Time: 3.22 minutes
Pandas Calculated Median Listening Time: 3.22 minutes


The median tells me most songs I listen to are about 3 minutes long. That’s perfect for someone like me who’s always coding or working out. Short tracks mean I can enjoy a quick music break without losing focus on what I’m doing.


### Range


In [14]:

# Calculate the range of listening times to see the spread of track lengths

# Find the shortest and longest track
min_time = all_streaming_df['minutesPlayed'].min()
max_time = all_streaming_df['minutesPlayed'].max()

# Calculate the range
range_of_listening = max_time - min_time

print(f"Shortest Track: {min_time:.2f} minutes")
print(f"Longest Track: {max_time:.2f} minutes")
print(f"Listening Time Range: {range_of_listening:.2f} minutes")


Shortest Track: 0.00 minutes
Longest Track: 18.81 minutes
Listening Time Range: 18.81 minutes


This shows the variety in my music selection, from quick tunes that barely hit the minute mark to epic tracks for when I'm deep into coding or just chilling. It’s interesting to see such a wide range but how come I have listened to a 0.00 minutes song 😂 ? let's see how many they are. maybe my data is not as clean as I thought

In [15]:
num_records_less_than_zero = len(all_streaming_df[all_streaming_df['minutesPlayed'] <= 0.0])
print(f"Number of records with songs less than 0.0 minutes: {num_records_less_than_zero}")


Number of records with songs less than 0.0 minutes: 96


What ?😅 Now I'm very curious what those songs are

In [16]:
# Define the variable "unique_zero_minutes_records" before using it
unique_zero_minutes_records = all_streaming_df[all_streaming_df['minutesPlayed'] <= 0.0]

# Calculate the count of zero minutes records by artist and track
count_zero_minutes_records = unique_zero_minutes_records.groupby(['artistName', 'trackName']).size().reset_index(name='count')

print(count_zero_minutes_records)


             artistName                               trackName  count
0                 .Sinh                                Bergamot      1
1                  2Pac                           Ghetto Gospel      1
2                  2Pac  Thugz Mansion - 2Pac Original/Acoustic      1
3   5 Seconds of Summer                                   Teeth      1
4               50 Cent                          Just A Lil Bit      1
..                  ...                                     ...    ...
88    Twenty One Pilots                                    Ride      1
89            Vance Joy                        We're Going Home      1
90             YeloHill       Judas (feat. Lecrae) - Open Verse      1
91           Youssoupha                                 PÉTROLE      1
92           daigoro789             Natsuhibshi (From "Naruto")      1

[93 rows x 3 columns]


Okay these are real songs with real artist 😂
I think these are songs I actually skipped, let's see the length of the play in milliseconds and that may prove my point

In [17]:
all_streaming_df[all_streaming_df["msPlayed"]<=0]

,endTime,artistName,trackName,msPlayed,minutesPlayed,dayOfWeek,hourOfDay
128,2023-03-13 12:51:00,Chuck Berry,Route 66,0,0.0,Monday,12
280,2023-03-14 10:26:00,92 Keys,How Far I'll Go,0,0.0,Tuesday,10
456,2023-03-16 11:56:00,Foo Fighters,Let It Die,0,0.0,Thursday,11
525,2023-03-17 19:29:00,Sauti Sol,Short N Sweet,0,0.0,Friday,19
1679,2023-03-27 12:22:00,Fitz and The Tantrums,Out of My League,0,0.0,Monday,12
...,...,...,...,...,...,...,...
26385,2024-02-25 14:07:00,Les Archanges,Jésus Règne,0,0.0,Sunday,14
26386,2024-02-25 14:07:00,Les Archanges,Elevons Jesus,0,0.0,Sunday,14
27365,2024-03-05 18:02:00,Kaaris,Angle mort,0,0.0,Tuesday,18
27369,2024-03-05 18:12:00,Kaaris,Sur le banc,0,0.0,Tuesday,18


Wow they really are absolute 0, I'll do what I learned in the first Learning from data class after recess
I'll just look for those songs in my dataset and look at them with my own eyes.
it's over 90 records of them though, I still believe that they may have been skipped I do not see any other reason why.
I would love to hear your thought about this Sir. quick email: <heritierkaumbu@icloud.com>

### Variance

In [18]:
# Calculate mean listening time
mean_listening_time = all_streaming_df['minutesPlayed'].mean()

# Calculate squared differences from the mean
squared_diffs = (all_streaming_df['minutesPlayed'] - mean_listening_time) ** 2

# Manually calculate the variance
variance_manual = squared_diffs.mean()

print(f"Manually Calculated Variance: {variance_manual:.2f} minutes squared")

# Using Pandas to calculate the variance
variance_pandas = all_streaming_df['minutesPlayed'].var()
print(f"Pandas Calculated Variance: {variance_pandas:.2f} minutes squared")


Manually Calculated Variance: 2.89 minutes squared
Pandas Calculated Variance: 2.89 minutes squared


### Standard Deviation

In [19]:
# Manual calculation of standard deviation
# Variance was previously calculated manually as variance_manual
std_dev_manual = variance_manual ** 0.5

print(f"Manually Calculated Standard Deviation: {std_dev_manual:.2f} minutes")

# Using Pandas to calculate the standard deviation
std_dev_pandas = all_streaming_df['minutesPlayed'].std()
print(f"Pandas Calculated Standard Deviation: {std_dev_pandas:.2f} minutes")


Manually Calculated Standard Deviation: 1.70 minutes
Pandas Calculated Standard Deviation: 1.70 minutes


The std is not that far from the average, meaning my track lengths don't swing wildly.

### Kurtosis

In [22]:
# Calculate mean and standard deviation
mean_listening_time = all_streaming_df['minutesPlayed'].mean()
sigma = all_streaming_df['minutesPlayed'].std()
n = len(all_streaming_df['minutesPlayed'])

# Calculate the sum of the fourth power of deviations from the mean
sum_fourth_power_deviations = ((all_streaming_df['minutesPlayed'] - mean_listening_time) ** 4).sum()

# Apply the formula for kurtosis
kurtosis_manual = (n * (n + 1) / ((n - 1) * (n - 2) * (n - 3))) * (sum_fourth_power_deviations / sigma ** 4) - (3 * (n - 1) ** 2 / ((n - 2) * (n - 3)))

# Using Pandas to calculate kurtosis for comparison
kurtosis_ = all_streaming_df['minutesPlayed'].kurtosis()

print(f"Manually Calculated Kurtosis: {kurtosis_manual:.2f}")
print(f"Pandas Calculated Kurtosis: {kurtosis_:.2f}")


Manually Calculated Kurtosis: 2.26
Pandas Calculated Kurtosis: 2.26
